In [1]:
import os
import pickle

import skimage
from skimage import data
import numpy as np
from numpy import matlib
from sklearn.datasets import load_svmlight_file
from PIL import Image
import matplotlib.pylab as plt
from matplotlib.pyplot import imshow

import scfw.poisson as ps
from scfw.scopt import scopt
from scfw.prox_grad import prox_grad

In [2]:
%matplotlib inline

## Data load

In [3]:
data_folder = '../data'

problem_name = 'a1a'
results = {problem_name: {}}
W, y = load_svmlight_file(os.path.join(data_folder, problem_name))
y = y.reshape(-1, 1)
W = W.toarray()

results_folder = './results/'
results_file=os.path.join(results_folder, 'poisson', problem_name+'.pckl')
results = {problem_name: {}}
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        results=pickle.load(f)

## Data preprocessing

In [4]:
# multiplicative multiplication of the structure of the recognizable structure
W = matlib.repmat(y, 1, W.shape[1]) * W
# summation of feature descriptions
sA = np.sum(W, 1)
# if the sum of the string is negative, invert it
W[sA < 0, :] = -W[sA < 0, :]
# zero row deletion
W = W[sA != 0, :]
# W = scipy.sparse.csr_matrix(W)

Bias = 1
b = (Bias * y).squeeze()
b = np.abs(b)
if (b.any == 0):
    print(' Input parameter y error')

y = b
N, n = W.shape

## Parameters initialization

In [5]:
lam =  np.sqrt(N)/2
#first set of parameters
Mf = max(2 / np.sqrt(y))
nu = 3
I = np.eye(n)
Iv = np.ones(n)
#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10
M = ps.poisson_matr(W, y, lam, x0)[0] / lam

FW_params={
    'iter_FW': 50000,
    'line_search_tol': 1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Auxilary functions

In [6]:
func_x = lambda x: ps.poisson_matr(W, y, lam, x)
grad_x = lambda x, dot_product: ps.grad_poisson_matr(W, y, lam, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: ps.grad_poisson_matr(W, y, lam, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_mult_x = lambda x, dot_product: ps.hess_mult_matr(W, y, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: ps.hess_mult_vec_matr(W, y, s, dot_product)
extra_func = lambda x: W @ x
linear_orcale = lambda grad: ps.linear_oracle_full_simplex(grad, M)
prox_func = lambda x, L: np.maximum(x, 0) #used for SCOPT

## Run SCOPT

In [7]:
x, alpha_hist, Q_hist, time_hist = scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          x0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=10)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    #'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
}

40.16
Fista err = 3.977e-09; Subiter =   2; subproblem converged!
Fista err = 3.919e-09; Subiter =   2; subproblem converged!
Fista err = 3.862e-09; Subiter =   2; subproblem converged!
Fista err = 3.805e-09; Subiter =   2; subproblem converged!
Fista err = 3.750e-09; Subiter =   2; subproblem converged!
Fista err = 3.695e-09; Subiter =   2; subproblem converged!
Fista err = 3.641e-09; Subiter =   2; subproblem converged!
Fista err = 3.588e-09; Subiter =   2; subproblem converged!
Fista err = 3.535e-09; Subiter =   2; subproblem converged!
Fista err = 3.484e-09; Subiter =   2; subproblem converged!
iter =  460, stepsize = 1.000e+00, rdiff = 6.993e-09 , f = 1640.16
Fista err = 3.433e-09; Subiter =   2; subproblem converged!
Fista err = 3.383e-09; Subiter =   2; subproblem converged!
Fista err = 3.333e-09; Subiter =   2; subproblem converged!
Fista err = 3.285e-09; Subiter =   2; subproblem converged!
Fista err = 3.237e-09; Subiter =   2; subproblem converged!
Fista err = 3.189e-09; Subi

## Run prox grad

In [9]:
def run_prox_grad(problem_name):
    results = {problem_name: {}}
    W, y = load_svmlight_file(os.path.join(data_folder, problem_name))
    y = y.reshape(-1, 1)
    W = W.toarray()
    results_file=os.path.join(results_folder, 'poisson', problem_name+'.pckl')
    results = {problem_name: {}}
    if os.path.isfile(results_file):
        with open(results_file, "rb") as f:
            results=pickle.load(f)
    # multiplicative multiplication of the structure of the recognizable structure
    W = matlib.repmat(y, 1, W.shape[1]) * W
    # summation of feature descriptions
    sA = np.sum(W, 1)
    # if the sum of the string is negative, invert it
    W[sA < 0, :] = -W[sA < 0, :]
    # zero row deletion
    W = W[sA != 0, :]
    # W = scipy.sparse.csr_matrix(W)
    Bias = 1
    b = (Bias * y).squeeze()
    b = np.abs(b)
    if (b.any == 0):
        print(' Input parameter y error')
    y = b
    N, n = W.shape
    lam =  np.sqrt(N)/2
    Mf = max(2 / np.sqrt(y))
    nu = 3
    I = np.eye(n)
    Iv = np.ones(n)
    x0 = np.ones(n) / n
    terminate_tol = 1e-10
    M = ps.poisson_matr(W, y, lam, x0)[0] / lam
    func_x = lambda x: ps.poisson_matr(W, y, lam, x)
    grad_x = lambda x, dot_product: ps.grad_poisson_matr(W, y, lam, x, dot_product)
    grad_beta = lambda x, s, beta, dot_product, dot_product_s: ps.grad_poisson_matr(W, y, lam, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
    hess_mult_x = lambda x, dot_product: ps.hess_mult_matr(W, y, x, dot_product)
    hess_mult_vec_x = lambda s, dot_product: ps.hess_mult_vec_matr(W, y, s, dot_product)
    extra_func = lambda x: W @ x
    linear_orcale = lambda grad: ps.linear_oracle_full_simplex(grad, M)
    prox_func = lambda x, L: np.maximum(x, 0) #used for SCOPT
    prox_params={
        #parameters for SCOPT
        'iter_prox': 50000,
        'Lest': 'estimate',#,'estimate', #estimate L
        'bb_type': 2,
        #FISTA parameters
        'fista_type': 'fista',
        'fista_tol': 1e-5,
        'fista_iter': 1000,
        'btk_iters': 100,
        'backtracking': True
    }

    x, alpha_hist, Q_hist, time_hist = prox_grad(func_x,
                grad_x,
                hess_mult_vec_x,
                prox_func,
                Mf,
                x0,
                prox_params,
                eps=terminate_tol,
                print_every=1000)

    results[problem_name]['prox_grad'] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)

In [11]:
data_list =  ['a1a','a2a','a3a', 'a4a', 'a5a', 'a6a','a7a','a8a','a9a']
for problem_name in data_list:
    run_prox_grad(problem_name)

size = 2.753e-05, rdiff = 4.800e-02 , f = 10105.1
iter = 11000, stepsize = 2.831e-05, rdiff = 4.898e-02 , f = 10008.3
iter = 12000, stepsize = 2.913e-05, rdiff = 5.000e-02 , f = 9909.49
iter = 13000, stepsize = 1.500e-05, rdiff = 1.021e-01 , f = 9808.61
iter = 14000, stepsize = 3.093e-05, rdiff = 5.216e-02 , f = 9705.58
iter = 15000, stepsize = 3.192e-05, rdiff = 5.331e-02 , f = 9600.31
iter = 16000, stepsize = 1.648e-05, rdiff = 1.090e-01 , f = 9492.7
iter = 17000, stepsize = 3.409e-05, rdiff = 5.575e-02 , f = 9382.67
iter = 18000, stepsize = 1.764e-05, rdiff = 1.141e-01 , f = 9270.1
iter = 19000, stepsize = 1.829e-05, rdiff = 1.168e-01 , f = 9154.9
iter = 20000, stepsize = 1.898e-05, rdiff = 1.196e-01 , f = 9036.95
iter = 21000, stepsize = 3.946e-05, rdiff = 6.128e-02 , f = 8916.14
iter = 22000, stepsize = 4.108e-05, rdiff = 6.282e-02 , f = 8792.32
iter = 23000, stepsize = 4.283e-05, rdiff = 6.443e-02 , f = 8665.38
iter = 24000, stepsize = 4.475e-05, rdiff = 6.611e-02 , f = 8535.17
i

In [7]:
prox_params={
        #parameters for SCOPT
        'iter_prox': 50000,
        'Lest': 'estimate',#,'estimate', #estimate L
        'bb_type': 1,
        #FISTA parameters
        'fista_type': 'fista',
        'fista_tol': 1e-5,
        'fista_iter': 1000,
    }

x, alpha_hist, Q_hist, time_hist = prox_grad(func_x,
            grad_x,
            hess_mult_vec_x,
            prox_func,
            Mf,
            x0,
            prox_params,
            eps=terminate_tol,
            print_every=1000)

results[problem_name]['prox_grad'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
}

iter =    1, stepsize = 6.343e-05, rdiff = 1.592e+01 , f = 3658.74
iter = 1000, stepsize = 7.845e-05, rdiff = 9.239e-02 , f = 3452.16
iter = 2000, stepsize = 9.291e-05, rdiff = 1.043e-01 , f = 3256.85
iter = 3000, stepsize = 1.139e-04, rdiff = 1.192e-01 , f = 3035.15
iter = 4000, stepsize = 1.470e-04, rdiff = 1.384e-01 , f = 2779.97
iter = 5000, stepsize = 2.070e-04, rdiff = 1.626e-01 , f = 2482.21
iter = 6000, stepsize = 3.493e-04, rdiff = 1.883e-01 , f = 2134.04
iter = 7000, stepsize = 1.096e-03, rdiff = 1.634e-01 , f = 1765.1
iter = 8000, stepsize = 1.000e+00, rdiff = 4.605e-07 , f = 1640.16
iter = 9000, stepsize = 1.000e+00, rdiff = 1.815e-08 , f = 1640.16
iter = 10000, stepsize = 1.000e+00, rdiff = 4.908e-09 , f = 1640.16
iter = 11000, stepsize = 1.000e+00, rdiff = 2.519e-09 , f = 1640.16
iter = 12000, stepsize = 1.000e+00, rdiff = 1.205e-09 , f = 1640.16
iter = 13000, stepsize = 1.000e+00, rdiff = 1.168e-09 , f = 1640.16
iter = 14000, stepsize = 1.000e+00, rdiff = 7.506e-10 , f =

In [8]:
with open(results_file, 'wb') as f:
    pickle.dump(results, f)

In [8]:
results['a1a'].keys()

dict_keys(['standard', 'line_search', 'backtracking', 'sc', 'sc_backtracking', 'sc_hybrid', 'scopt', 'prox_grad'])